In [1]:
from dwave.cloud import Client
from pyqubo import Array, Constraint, Placeholder, solve_qubo, Sum, Binary
import dimod
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
import dwave
import random
from pyqubo import Binary
import requests
import json
import re

In [2]:
#  D'Wave acccess key
token = 'WASE-c2ee8615ffb9fc1953570636217c253aa213dc34'
client = Client.from_config(token=token)
# solverDW = 'DW_2000Q_6' 
solverDW = 'DW_2000Q_VFYC_6'
# ---------------------------
#  Fujitsu DA acccess key
post_url = 'https://api.aispf.global.fujitsu.com/da/v2/qubo/solve'
post_headers = {'X-Api-Key' : 'aac938c03db1254b7b1cef3f6961eb9bf101ade2fe8aaa4b3b322d5d11f43793', \
                   'Accept': 'application/json', \
                   'Content-Type': 'application/json'}
# ---------------------------

In [26]:
def get_random():
    while True:
        x=random.randint(-1,1)
        if x == -1 or x == 1:
            return x
        
def make_hami_model_seho(N):
    x = Array.create('x', (N, N), 'BINARY')
    cons = 0
    for i in range(N):
        for j in range(N-1):
                cons += get_random() * x[i][j] * x[i][j+1] ### ここでJ_ij を変える
    for i in range(N-1):
        for j in range(N):
                cons += get_random() * x[i][j] * x[i+1][j]

    j_constraint = Constraint(cons, label='seho')
    H =j_constraint
    # Compile model
    model = H.compile()
    return model

def make_hami_model_seho_bunk(N):
    if (N > 8):
        m = N % 8
        count = 0
        newN = N - m
        models = []
        while(newN > 1):
            newN = newN - 8
            x = Array.create('x', (8, 8), 'BINARY')
            cons = 0
            for i in range(8):
                for j in range(7):
                    cons += get_random() * x[i][j] * x[i][j+1] ### ここでJ_ij を変える
            for i in range(7):
                for j in range(8):
                    cons += get_random() * x[i][j] * x[i+1][j]

            j_constraint = Constraint(cons, label='seho')
            H = j_constraint
            # Compile model
            model = H.compile()
            models.append(model)
    return models

def make_hami_model_ripo(N): 
    x = Array.create('x', (N, N*N), 'BINARY')
    cons = 0
    for k in range(N):
        for i in range(N):
                for j in range(N-1):
                    cons += -1 * x[i][k*N+j] * x[i][k*N+j+1] ### ここでJ_ij を変える
                    
    for k in range(N):
        for i in range(N-1):
                for j in range(N):
                    cons += -1 * x[i][k*N+j] * x[i+1][k*N+j]
                    
    for k in range(N-1):
        for i in range(N):
                for j in range(N):
                    cons += -1 * x[i][k*N+j] * x[i][(k+1)*N+j]

    j_constraint = Constraint(cons, label='ripo')
    H =j_constraint
    # Compile model
    model = H.compile()
    return model

def make_hami_model_rand(N, p):
    x = Array.create('x', (N, 1), 'BINARY')
    cons = 0
    for i in range(N):
        for j in range(i, N):
            if p > random.random():
                cons += -1 * x[i][0] * x[j][0] ### ここでJ_ij を変える. Jij の係数が全て -1なら答えは全部 True

    j_constraint = Constraint(cons, label='rand')
    H =j_constraint
    # Compile model
    model = H.compile()
    return model

def make_hami_model_rand_bunk(N, p):
    if (N > 64):
        m = N % 64
        newN = N - m
        models = []
        while(newN > 1):
            newN = newN - 64
            x = Array.create('x', (64, 1), 'BINARY')
            cons = 0
            for i in range(64):
                for j in range(i, 64):
                    if p > random.random():
                        cons += random.random() * x[i][0] * x[j][0] ### ここでJ_ij を変える. Jij の係数が全て -1なら答えは全部 True

            j_constraint = Constraint(cons, label='rand')
            H =j_constraint
    # Compile model
            model = H.compile()
            models.append(model)
    return models

def PyQdic_to_DwaveQubo(qdic, offset, shape):           
    # pyqubo x0, x1 to Fujitsu DA BinaryPolynomial
    # "shape" should be same as "(N,K) in Array.create('x', (N, K), 'BINARY')"
    dwdic = {}
    N, M = shape # row, col
    for k in qdic:
        x0, x1 = k
        s = list(map(lambda x: int(x[1:][:-1]), re.findall('\[\d+\]', x0)))
        t = list(map(lambda x: int(x[1:][:-1]), re.findall('\[\d+\]', x1)))
        ns = M * s[0] + s[1]
        nt = M * t[0] + t[1]
#         print(k, ns, nt)
        dwdic[(ns, nt)] = qdic[k]
    return dwdic

def DwaveSol_to_PyqSol(sol, shape):
    N, M = shape # row, col
    PyQ_res = dict()
    for n in range(N):
        for m in range(M):
            i = M * n + m
            PyQ_res['x[{}][{}]'.format(n, m)] = int(sol[i])
    return PyQ_res

def calc_anneal_schedule(sp, tp):
    time = [0.0, sp, sp + tp, 2*sp + tp] # unit in [us]
    flux = [0.0, 0.5, 0.5, 1.0]
    if sp == 0:
        print('invalid sp!!')
        sys.exit(1)
    ar = len(flux)
    schedule = [[time[i], flux[i]] for i in range(ar)]
    return schedule

def PyQdic_to_Fdic(qdic, offset, shape, anneal_arg):           
    # pyqubo x0, x1 to Fujitsu DA BinaryPolynomial
    # "shape" should be same as "(N,K) in Array.create('x', (N, K), 'BINARY')"
    fdic = {}
    gdic = {}
    alist = []
    N, M = shape # row, col
    for k in qdic:
        edic = {}
        x0, x1 = k
        s = list(map(lambda x: int(x[1:][:-1]), re.findall('\[\d+\]', x0)))
        t = list(map(lambda x: int(x[1:][:-1]), re.findall('\[\d+\]', x1)))
        if len(s) == 1:
            a0 = s[0]
            a1 = t[0]
        else:
            a0 = M * s[0] + s[1]
            a1 = M * t[0] + t[1]
        edic["coefficient"] = qdic[k]
        edic["polynomials"] = [int(a0), int(a1)]
        alist.append(edic)
    edic = {}
    edic["coefficient"] = offset
    alist.append(edic)
    gdic["terms"] = alist
    fdic["binary_polynomial"] = gdic 
    ddic = anneal_arg
    fdic["fujitsuDA2PT"] = ddic #fujitsuDAPT, fujitsuDA2PT などがある。
    return fdic

def req(ddic):
    #POSTパラメータは二つ目の引数に辞書で指定する 
    response = requests.post(post_url,
            json.dumps(ddic), \
            headers=post_headers)
    print(response.json())
    print(type(response.json()))
    return response

In [27]:
N = 128 # seho -> 8まで, ripo -> 4まで, rand -> 64まで
n = N #seho N ** 2, ripo N ** 3, rand N
if N > 64:
    size = (64,1)
    n = 64
else:
    size = (N, 1)
model = make_hami_model_rand_bunk(N,0.4)
results = []
for i in range(len(model)):
    qubo, offset = model[i].to_qubo()
    dwQubo = PyQdic_to_DwaveQubo(qubo, offset, size) # Dwave のモデルを作る

    sampler = DWaveSampler(token=token, solver=solverDW) # Dwave の設定 & 実行
    _hardware_adj = sampler.adjacency
    chain_st = 3.2
    _nSol = 1
    sp = 120
    tp = 220
    schedule = calc_anneal_schedule(sp, tp)
    el = sampler.edgelist
    _embedding = dwave.embedding.chimera.find_clique_embedding(n, 16, 16, 4, target_edges=el)
    embQubo = dwave.embedding.embed_qubo(dwQubo, _embedding, _hardware_adj, chain_strength=chain_st)
    res_list = list()
    for i in range(_nSol):
        _response = sampler.sample_qubo(embQubo, num_reads=_nSol,
                                    readout_thermalization=100,
                                    programming_thermalization=100,
                                    anneal_schedule=schedule)
        bqm = dimod.BinaryQuadraticModel.from_qubo(dwQubo, offset=offset)
        response = dwave.embedding.unembed_sampleset(_response, _embedding, bqm,
                                                     chain_break_method=None)
        res_list.append(response.record)

    z=response.first.sample
    zz=DwaveSol_to_PyqSol(z, size) # Dwave の解を デコード
    results.append(zz)

In [32]:
    sp = 120
    tp = 220
    schedule = calc_anneal_schedule(sp, tp)
    print(schedule)

[[0.0, 0.0], [120, 0.5], [340, 0.5], [460, 1.0]]


In [28]:
results
print(len(model))

2


In [29]:
inputs = []
for i in range(len(results)):
    inputs.append(list(results[i].values()))
import itertools
DAinputs = list(itertools.chain.from_iterable(inputs))
toFjtsuLis = list(DAinputs) # dwave の解を DA の初期値へ list() とすればDAの初期値はランダム
config = dict()
ind = 1
for i in toFjtsuLis:
    config[str(ind)] = bool(i)
    ind = ind + 1
# DA のパラメータ
anneal_arg = {
        "expert_mode": True,
        "noise_model": "METROPOLIS",
        "number_iterations": int(200),
        "number_runs": int(128),
        "offset_increase_rate": int(1e2),
        "temperature_decay": 1e-3,
        "temperature_interval": int(50),
        "temperature_mode": "INVERSE_ROOT",
        "temperature_start": 10,
        "solution_mode": "COMPLETE",
        "guidance_config": config
    }
fuji_dic = PyQdic_to_Fdic(qubo, offset, size, anneal_arg) # Pyqubo を fujitsu 用へ
resdic = req(fuji_dic) #jsonを送信。あとはデコード
res_dic = resdic.json()
res_keys = [str(num) for num in range(N)]
# print(res_dic['qubo_solution']['solutions'][0]['configuration'][res_keys[0]])
# print(res_dic['qubo_solution']['solutions'][0]['energy'])
e = 0
fre = 0
for i in range(len(res_dic['qubo_solution']['solutions'])):
    e = e + res_dic['qubo_solution']['solutions'][i]['energy']
    fre = fre + res_dic['qubo_solution']['solutions'][i]['frequency']
print(e/len(res_dic['qubo_solution']['solutions']))
print(fre/len(res_dic['qubo_solution']['solutions']))

{'qubo_solution': {'result_status': True, 'solutions': [{'energy': 0, 'frequency': 1, 'configuration': {'0': False, '1': False, '2': False, '3': False, '4': False, '5': False, '6': False, '7': False, '8': False, '9': False, '10': False, '11': False, '12': False, '13': False, '14': False, '15': False, '16': False, '17': False, '18': False, '19': False, '20': False, '21': False, '22': False, '23': False, '24': False, '25': False, '26': False, '27': False, '28': False, '29': False, '30': False, '31': False, '32': False, '33': False, '34': False, '35': False, '36': False, '37': False, '38': False, '39': False, '40': False, '41': False, '42': False, '43': False, '44': False, '45': False, '46': False, '47': False, '48': False, '49': False, '50': False, '51': False, '52': False, '53': False, '54': False, '55': False, '56': False, '57': False, '58': True, '59': False, '60': True, '61': False, '62': False, '63': False}}, {'energy': 0, 'frequency': 1, 'configuration': {'0': False, '1': False, '2

In [30]:
inputs = []
for i in range(len(results)):
    inputs.append(list(results[i].values()))
import itertools
DAinputs = list(itertools.chain.from_iterable(inputs))
toFjtsuLis = list() # dwave の解を DA の初期値へ list() とすればDAの初期値はランダム
config = dict()
ind = 1
for i in toFjtsuLis:
    config[str(ind)] = bool(i)
    ind = ind + 1
# DA のパラメータ
anneal_arg = {
        "expert_mode": True,
        "noise_model": "METROPOLIS",
        "number_iterations": int(200),
        "number_runs": int(128),
        "offset_increase_rate": int(1e2),
        "temperature_decay": 1e-3,
        "temperature_interval": int(50),
        "temperature_mode": "INVERSE_ROOT",
        "temperature_start": 10,
        "solution_mode": "COMPLETE",
        "guidance_config": config
    }
fuji_dic = PyQdic_to_Fdic(qubo, offset, size, anneal_arg) # Pyqubo を fujitsu 用へ
resdic = req(fuji_dic) #jsonを送信。あとはデコード
res_dic = resdic.json()
res_keys = [str(num) for num in range(N)]
# print(res_dic['qubo_solution']['solutions'][0]['configuration'][res_keys[0]])
# print(res_dic['qubo_solution']['solutions'][0]['energy'])
e = 0
fre = 0
for i in range(len(res_dic['qubo_solution']['solutions'])):
    e = e + res_dic['qubo_solution']['solutions'][i]['energy']
    fre = fre + res_dic['qubo_solution']['solutions'][i]['frequency']
print(e/len(res_dic['qubo_solution']['solutions']))
print(fre/len(res_dic['qubo_solution']['solutions']))

{'qubo_solution': {'result_status': True, 'solutions': [{'energy': 0, 'frequency': 1, 'configuration': {'0': False, '1': False, '2': False, '3': False, '4': False, '5': False, '6': False, '7': False, '8': False, '9': False, '10': False, '11': False, '12': False, '13': False, '14': False, '15': False, '16': False, '17': False, '18': False, '19': False, '20': False, '21': False, '22': False, '23': False, '24': False, '25': False, '26': False, '27': False, '28': False, '29': False, '30': False, '31': False, '32': False, '33': False, '34': False, '35': False, '36': False, '37': False, '38': False, '39': False, '40': False, '41': False, '42': False, '43': False, '44': False, '45': False, '46': False, '47': False, '48': False, '49': False, '50': False, '51': False, '52': False, '53': False, '54': False, '55': False, '56': True, '57': False, '58': False, '59': False, '60': False, '61': False, '62': False, '63': False}}, {'energy': 0, 'frequency': 1, 'configuration': {'0': False, '1': False, '